In [108]:
import os
import csv
from matplotlib import rc

In [109]:
base = 5
path = f'N={base}'

In [110]:
rc('font', size=12, **{'family': 'serif', 'serif': ['Times New Roman']})

def ordenar_por_numero(nombre_archivo):
    # Obtener el número del nombre del archivo
    numero = int(nombre_archivo.split('=')[1].split('.')[0])
    return numero

In [111]:
def format_number(number):
    # Convertir el número a una cadena con notación científica para extraer la primera cifra significativa
    scientific_str = f"{number:.1e}"
    first_digit = int(scientific_str[0])
    
    # Si la primera cifra significativa es 1, usar dos cifras significativas
    if first_digit == 1:
        formatted_number = f"{number:.2g}"
    else:
        # Si la primera cifra significativa es diferente de 1, usar una cifra significativa
        formatted_number = f"{number:.1g}"
    
    return formatted_number

def match_decimals(number_to_format, reference_number):
    # Formatear el número de referencia para determinar la cantidad de decimales
    formatted_reference = format_number(reference_number)
    
    # Extraer la parte decimal del número formateado
    if '.' in formatted_reference:
        decimals = len(formatted_reference.split('.')[1])
    else:
        decimals = 0
    
    # Formatear el número a formatear con la misma cantidad de decimales
    formatted_number = f"{number_to_format:.{decimals}f}"
    
    return formatted_number

In [112]:
# files = [file for file in os.listdir(f'{path}') if file.endswith('.csv')]
# for i, file in enumerate(files):
#     alpha  = []
#     energy = []

#     with open(f'{path}/{file}', newline='') as csvfile:
#         reader = csv.reader(csvfile)
#         next(reader)

#         for row in reader:
#             alpha.append(float(row[0]))
#             energy.append(float(row[1]))

#     for n in range(len(energy)-1):
#         if energy[n] * energy[n+1] < 0:
#             alpha_c_exact = round((alpha[n] + alpha[n+1]) / 2, 3)
#             print(alpha_c_exact)

alpha_c_exact = 1.19061074

In [113]:
folders = [folder for folder in os.listdir(path) if '.' not in folder]

for n, folder in enumerate(folders):
    files = [file for file in os.listdir(f'{path}/{folder}') if file.endswith('.csv')]
    files = sorted(files, key=ordenar_por_numero)

    for i, file in enumerate(files):
        N = int(file.split('=')[1].split('.')[0])

        alpha  = []
        energy_vqe = []

        with open(f'{path}/{folder}/{file}', newline='') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)

            for row in reader:
                alpha.append(float(row[0]))
                energy_vqe.append(float(row[1]))


        alpha_c = []

        for n in range(len(energy_vqe)-1):
            if energy_vqe[n] * energy_vqe[n+1] < 0:
                alpha_c.append((alpha[n] + alpha[n+1]) / 2)
        
        a = folder.split('_')[-1]
        b = file.split('=')[-1].split('.')[0]

        if len(alpha_c) == 0:
            print(f'			${a}$ & ${b}$ & & \\\\')
        elif len(alpha_c) == 1:
            c = alpha_c[0]
            d = 0.0005
            c = match_decimals(float(c), float(d))
            e = abs(round((float(c) - alpha_c_exact) / alpha_c_exact * 100, 2))
            print(f'			${a}$ & ${b}$ & ${c} \\pm {d}$ & ${e}\\%$ \\\\')
        else:
            c = (alpha_c[0] + alpha_c[-1]) / 2
            d = format_number(abs((alpha_c[0] - alpha_c[-1]) / 2))
            c = match_decimals(float(c), float(d))
            e = abs(round((float(c) - alpha_c_exact) / alpha_c_exact * 100, 2))
            print(f'			${a}$ & ${b}$ & ${c} \\pm {d}$ & ${e}\\%$ \\\\')

			$COBYLA$ & $16$ & $1.036 \pm 0.006$ & $12.99\%$ \\
			$COBYLA$ & $128$ & & \\
			$COBYLA$ & $1024$ & $1.13 \pm 0.01$ & $5.09\%$ \\
			$COBYLA$ & $16384$ & $1.121 \pm 0.002$ & $5.85\%$ \\
